模拟环境设计如下：
	•	价格选项（arms）：
[0.1, 0.2, 0.3, 0.4]，代表 Liquidity Provider 可提供的报价。
	•	成交概率（conversion rate）：
与价格成反比，越便宜越容易成交：[0.9, 0.6, 0.4, 0.2]
	•	奖励函数（reward）：
成交即得该价格利润，否则为 0。
	•	代理人数量（n_agents）：
设置为 10，模拟多个 LP 同时使用学习算法。
	•	轮数（T）：
每个 agent 运行 1000 轮选择任务，观察长期行为。

⸻算法	特点	适用环境
EXP3	基于概率抽样，适应性强	非平稳或对抗性环境
UCB	上置信界算法，快速确定最优	平稳、高效探索环境
UCB-V	加入方差估计，更敏感	高方差 reward 情况
UCB-Tuned	控制波动，调和探索与利用	较稳健，适合中等波动市场

每种算法都试图在 “探索（exploration）” 和 “利用（exploitation）” 之间达到平衡。


你为每种算法运行以下过程：
	1.	每个 agent 在 1000 轮内选择报价；
	2.	根据 reward 函数记录是否成交；
	3.	持续更新每个臂（价格）被选择的频率；
	4.	汇总所有 agent 的选择行为，得到“每轮每个臂被选择的概率”。


Panel A: EXP3
	•	每条线初期都接近 0.25（4 臂均等）；
	•	随时间逐渐偏向某一臂，但其他臂没有完全消失；
	•	说明：EXP3 保持持续探索，策略分布多样，避免集体“抱团”。

Panel B: UCB
	•	某一臂的概率迅速上升，其他臂降到接近 0；
	•	所有 agent 很快“达成共识”；
	•	说明：UCB 收敛速度快，但可能导致伪合谋（pseudo-collusion）。

Panel C: UCB-V
	•	收敛速度比 UCB 更快；
	•	初期波动大，说明它更敏感，探索强；
	•	说明：适合高波动收益环境，但更容易“扎堆”。

Panel D: UCB-Tuned
	•	收敛速度介于 UCB 和 EXP3 之间；
	•	平衡探索与利用，策略较稳健；
	•	说明：适合中等噪声市场，不易误判 suboptimal 臂。


问题设定：

4个定价选项（0.1, 0.2, 0.3, 0.4），对应转化率（0.9, 0.6, 0.4, 0.2）。

每次选择价格后，根据转化概率获得收益（成功返回价格，否则0）。

目标是最大化累计收益，最优选项是期望收益（价格×转化率）最高的0.2和0.3（均为0.12）。

算法实现：

UCB算法：

维护每个动作的选择次数n_k和累计奖励r_k。

选择UCB值（平均奖励 + 探索项）最大的动作。

探索项随着时间增加而减小，平衡探索与利用。

EXP3算法：

维护权重向量，根据权重计算选择概率（带探索因子γ）。

使用重要性采样估计奖励，更新选中动作的权重。

γ控制探索程度，γ越大探索越强。

模拟流程：

每个算法独立运行10次（n_agents=10），每次1000步。

记录每个时间步各动作的选择次数，计算平均概率。

累积平均概率反映算法的收敛过程。

结果可视化：

绘制两种算法各动作的选择概率随时间变化曲线（对数时间轴）。

理想情况下，算法应快速收敛到最优动作（0.2和0.3），但由于两者期望相同，可能呈现相近概率。

关键观察：

UCB：初期快速收敛，但因需持续探索，可能保持对两个最优动作的选择。

EXP3：探索因子γ=0.3导致更多探索，收敛较慢，但适应非平稳环境更强。

改进建议：

调整EXP3的γ参数（如随时间衰减）以平衡探索与利用。

增加运行次数（n_agents）或时间步（T）提高稳定性。

检查多最优情况下的算法行为，验证是否合理分配选择概率。

结论：该代码有效对比了UCB和EXP3在动态定价中的表现，UCB可能更快锁定最优选项，而EXP3更适合对抗性环境。实际应用需根据问题特性选择算法